In [2]:
import sqlite3
import csv, json
import os
from datetime import datetime
db = 'taxi.db'
tb_name = 'yellow_tripdata_JanApr'

number_of_zones = 265

time_series_ofZone = {}
time_gap = {}
for i in range(1, number_of_zones+1):
    for j in range(1, number_of_zones+1):
        time_gap[str(i)+'+'+str(j)] = {}


In [10]:
lookup_tb = {}
with open('../data/taxi+_zone_lookup.csv', newline='') as csvfile:
    k=0
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if k==0:
            k+=1
            continue
        if row!=[]:
            lookup_tb[row[0]] = {'Borough': row[1].strip('\"'), 'Zone': row[2].strip('\"')}

reverse_lookup = {}
with open('../data/taxi+_zone_lookup.csv', newline='') as csvfile:
    k=0
    spamreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    for row in spamreader:
        if k==0:
            k+=1
            continue
        if row!=[]:
            reverse_lookup[row[2].strip('\"')] = {'Borough': row[1].strip('\"'), 'Zone': row[0]}


In [225]:
conn = sqlite3.connect(db)
cur = conn.cursor()
cur.execute('SELECT lpep_pickup_datetime, lpep_dropoff_datetime, PULocationID, DOLocationID FROM ' + tb_name)
rows = cur.fetchall()

In [159]:
k=0
j=0
for row in rows:
    if not k%100000:
        print(k/100000)
    k+=1
    travel = row[2]+'+'+row[3]
    pickup_time = row[0]
    dropoff_time = row[1]
    
    time_pu = datetime.strptime(pickup_time, '%Y-%m-%d %H:%M:%S')
    time_do = datetime.strptime(dropoff_time, '%Y-%m-%d %H:%M:%S')

#     timegap
    gap = (time_do-time_pu).total_seconds()

#     round to minute
    time_pu = time_pu.replace(second = 0)
    
    
    
    if time_pu in time_gap[travel]:
        gap_sum = time_gap[travel][time_pu][0] + gap
        gap_num = time_gap[travel][time_pu][1]+1
        time_gap[travel][time_pu] = [gap_sum, gap_num]
    else:
        time_gap[travel][time_pu] = [gap, 1]
    





0.0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0
170.0
171.0
172.0
173.0
174.0
175.0
176.0
177.0
178.0
179.0
180.0
181.0
182.0
183.0
184.0


In [161]:
k=0
for zone in time_gap:
    time_of_zone = time_gap[zone]
    time_series_ofZone[zone] = {}
864    k+=1
    if not k%1000:
        print(k)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000


In [162]:
with open('time_gap_eachzone.json', 'w') as jsonfile:
    json.dump(time_series_ofZone, jsonfile)

In [5]:
with open('time_gap_eachzone.json') as data_file:    
    time_series_ofZone = json.load(data_file)

In [6]:
import pymongo

In [11]:
from pymongo import MongoClient

In [12]:
client = MongoClient('localhost', 27017)

In [175]:
# table time_gap
time_gap = client.timegap
collection = time_gap.timeGap
k=0
for i in range(1, num_of_zones+1):
    for j in range(1, num_of_zones+1):
        zone_traveled = str(i)+'+'+str(j)
        ts = time_series_ofZone[str(i)+'+'+str(j)]
        ts_insert = {'travel': zone_traveled, 'values': ts}
        collection.insert_one(ts_insert).inserted_id
        k+=1
        if not k%1000:
            print(k)
result = collection.create_index([('travel', pymongo.ASCENDING)], unique=True)

In [18]:
# table traffic_travel
time_gap = client.timegap
collection = time_gap.traffic_travel

In [13]:
conn = sqlite3.connect(db)
cur = conn.cursor()
cur.execute('SELECT PULocationID, DOLocationID FROM ' + tb_name)
rows = cur.fetchall()

In [15]:
traffic_dict = {}
for i in range(1, number_of_zones+1):
    for j in range(1, number_of_zones+1):
        traffic_dict[str(i)+'+'+str(j)] = 0
        
for row in rows:
    travel = row[0] + '+' + row[1]
    traffic_dict[travel] += 1


In [19]:
k=0
for travel in traffic_dict:
    num = traffic_dict[travel]
    collection.insert_one({'travel': travel, 'traffic': num}).inserted_id
    if not k%10000:
        print(k)
    k+=1


0
10000
20000
30000
40000
50000
60000
70000
